In [12]:
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
import io
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output

In [13]:
model = tf.keras.models.load_model('tesla_road_model.keras')
IMG_SIZE = 256

In [14]:
@tf.function
def fast_predict(input_data):
    return model(input_data, training=False)

In [15]:
upload_btn = widgets.FileUpload(accept='image/*', multiple=False)
output = widgets.Output()

def process_upload(change):
    with output:
        clear_output()
        if not upload_btn.value: return
            
        file_info = upload_btn.value[0]
        img = Image.open(io.BytesIO(file_info['content']))
        img_np = np.array(img)
        if img_np.shape[-1] == 4: img_np = cv2.cvtColor(img_np, cv2.COLOR_RGBA2RGB)
        h, w, _ = img_np.shape

        # Preprocessing
        input_data = cv2.resize(img_np, (IMG_SIZE, IMG_SIZE)) / 255.0
        input_data = np.expand_dims(input_data.astype(np.float32), axis=0)
        input_tensor = tf.convert_to_tensor(input_data)

        # Inference (Warm-up-siz, oddiyroq usulda)
        start_time = time.perf_counter()
        prediction = fast_predict(input_tensor)
        end_time = time.perf_counter()
        
        inference_time = end_time - start_time

        # Vizualizatsiya
        mask = (prediction.numpy()[0] > 0.5).astype(np.uint8)
        mask_resized = cv2.resize(mask, (w, h))

        overlay = img_np.copy()
        overlay[mask_resized == 1] = [0, 255, 128]
        final_result = cv2.addWeighted(img_np, 0.7, overlay, 0.3, 0)

        print(f"✅ M4 Inference: {inference_time*1000:.2f} ms")
        plt.figure(figsize=(10, 5))
        plt.imshow(final_result)
        plt.axis('off')
        plt.show()

In [16]:
upload_btn.observe(process_upload, names='value')
display(upload_btn, output)

FileUpload(value=(), accept='image/*', description='Upload')

Output()